In [1]:
import numpy as np
import pandas as pd 
import sys, os, psutil, io
import gzip
import time 
import warnings

from pybedtools import BedTool
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

In [2]:
synteny_condensed=pd.read_csv('/pollard/home/shzhang/data/synteny/hg38_mm10_condensed.txt', sep='\t')

In [3]:
synteny_condensed

,hg_chr,hg_start,hg_end,hg_len,hg_seq,mm_chr,mm_start,mm_end,mm_len,mm_seq,total_align_len
0,chr1,56554,58923,2370,CACATTATTCTGCAGTTTTTATCTTGCTTACGTATTTTTGGAATGT...,chr2,111473169,111475499,2331,cacattgtattgta--ctctattatgttt-tattgtattatattat...,2518
1,chr1,60916,61124,209,TTAATATTTCCCACTTCATCATTACTAACCTCACTCACTCTGAACT...,chr2,111476095,111476298,204,ttAATAT--CCCAGTGCACC-TCACTACTCTGGCTCACGCTGAGCC...,209
2,chr1,61125,61835,711,AGTTCAAGACGTTTAATCAAGACCCTTCACTTACTTAACTATACCT...,chr2,111477065,111477685,621,aGCCCAAGAAAATTAAACAAATCCCTTTACTAATTTAAATGTATCT...,731
3,chr1,61943,61971,29,ctccgcctcccggggtcaagctattctcc,chr2,111477686,111477713,28,CTCTGCTTTGTGTGGAGAAG-TATTGTCC,29
4,chr1,62160,62467,308,tgtaaaactttttcctaatttaacagaaaaataatagtattatatt...,chr2,111477762,111478104,343,tgtaa----tttttcttatttaaCA-AGAAATGAGA-------ATT...,372
...,...,...,...,...,...,...,...,...,...,...,...
1229834,chrY,57107075,57107468,394,TGATTATGTTCGTTGTGACTGACAGCTGGATTGtagtggttaagaa...,chrX_GL456233_random,316168,316535,368,TAATGATGTTGATTGTGACTAGCAACAAGACTGtagtgattaagag...,397
1229835,chrY,57125268,57126152,885,aaagctgcagtgagtcgtaattgtgccactgcactccagcttgggc...,chrX_GL456233_random,321571,322437,867,aaggctgtatcg---cctaatagtgccacttcccatgagccaagca...,954
1229836,chrY,57126153,57126418,266,AGGTTGTTACTGCAAATCATTCATGTAGTCAGTGCCAGGGGTTAAA...,chrX_GL456233_random,322699,322963,265,ATATTGTTACTACAAATCATTCATATAGTCAGTGTGAAGAATAAAA...,266
1229837,chrY,57126419,57126561,143,CTTTTAATAGTGAAACACATGACCAGGCAGTGGC-TTTCCTATGAA...,chrX_GL456233_random,323295,323435,141,CTTTTCATCATAAAGTACATGACCAGGAAATATCTTTTCTTGTGAA...,150


In [5]:
#check that lengths of seqs and alignments are correct  
synteny_condensed['hg_dash_n']=synteny_condensed['hg_seq'].str.count('-')
synteny_condensed['mm_dash_n']=synteny_condensed['mm_seq'].str.count('-')

synteny_condensed['hg_percent']=1-(synteny_condensed['hg_dash_n']/synteny_condensed['total_align_len'])
synteny_condensed['mm_percent']=1-(synteny_condensed['mm_dash_n']/synteny_condensed['total_align_len'])


print(((synteny_condensed['hg_len'] + synteny_condensed['hg_dash_n']) == synteny_condensed['total_align_len']).all())
print(((synteny_condensed['mm_len'] + synteny_condensed['mm_dash_n']) == synteny_condensed['total_align_len']).all())

True
True


In [6]:
np.median(synteny_condensed['hg_percent'])

0.9612903225806452

In [7]:
chrom_len=pd.read_csv('/pollard/home/shzhang/akita_variant_scoring/data/chrom_lengths_hg38', sep='\t')

In [11]:
data_path='/pollard/data/projects/kgjoni/CBTN_collab/CBTN_data/'
scores=pd.read_csv(data_path + 'simple-variants_230620/simple_variants_scored/simple_variants_all_scores', sep='\t')
scores['oneminus_corrmean']=1-scores['corr_mean']
scores

,var_index,mse_mean,corr_mean,oneminus_corrmean
0,0,NaN,NaN,NaN
1,1,5.914621e-03,0.979139,2.086145e-02
2,2,1.629834e-07,1.000000,4.470926e-07
3,3,1.445605e-08,1.000000,3.242280e-08
4,4,2.073410e-08,0.999999,9.900652e-07
...,...,...,...,...
214039,214039,4.707555e-08,1.000000,9.518438e-08
214040,214040,NaN,NaN,NaN
214041,214041,NaN,NaN,NaN
214042,214042,NaN,NaN,NaN


In [ ]:
#get mse and corr idx 
mse_sort=scores.sort_values(by='mse_mean', ascending=False)
top_1000_mse = mse_sort.index[:500]
top_1000_mse_idx = mse_sort['var_index'].loc[top_1000_mse].tolist()

corr_sort=scores.sort_values(by='oneminus_corrmean', ascending=False)
top_1000_corr = corr_sort.index[:500]
top_1000_corr_idx = corr_sort['var_index'].loc[top_1000_corr].tolist()


In [9]:
top_variants=pd.read_csv('/pollard/home/shzhang/top_simple_variants', sep='\t')
top_variants

,overall_index
0,74790
1,129853
2,130218
3,130728
4,130730
...,...
137,182222
138,183088
139,108979
140,109077


In [ ]:
idx_union= list(set(top_1000_mse_idx).union(top_1000_corr_idx))
len(idx_union)

In [12]:
simple_df=pd.read_csv(data_path + 'simple-variants_230620/simple_variants_all', sep='\t')

In [14]:
#keep only those in top mse and corr
#simple_top = simple_df[simple_df['sample_var_index'].isin(idx_union)]
simple_top=simple_df[simple_df.index.isin(top_variants['overall_index'])]
simple_top['overall_index']=simple_top.index
simple_top

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,BiospecimenID,sample_var_index,overall_index
2425,chr10,122357800,.,CCA,C,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intergenic_variant|...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,1452,2425
2815,chr15,28432799,.,CAT,C,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intergenic_variant|...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,1842,2815
4993,chr2,32439175,.,TTG,T,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intron_variant|MODI...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A94A7EJ4,1501,4993
6422,chr3,17630126,.,AAGAGG,A,.,PASS,"CAL=Strelka2,Mutect2,VarDict,Lancet;CSQ=-|intr...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A94A7EJ4,2930,6422
10525,chr7,158180223,.,C,CAT,.,PASS,"CAL=Strelka2,Mutect2,Lancet;CSQ=AT|intron_vari...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A94A7EJ4,7033,10525
...,...,...,...,...,...,...,...,...,...,...,...,...
206438,chr9,88136504,.,C,CTT,.,PASS,"CAL=Strelka2,VarDict;CSQ=TT|intron_variant|MOD...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_G6V64VWY,1338,206438
210546,chr10,46535047,.,G,GCC,.,PASS,"CAL=Strelka2,Mutect2,VarDict;CSQ=CC|intron_var...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_QR84QJN4,3072,210546
211562,chr15,28646249,.,ACTTTTC,A,.,PASS,"CAL=Mutect2,VarDict;CSQ=-|downstream_gene_vari...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_QR84QJN4,4088,211562
212556,chrX,85899908,.,TAACAAGTGTTGATGAGGATACAGAAAAAAGCAGACCCTTTACACT...,T,.,PASS,"CAL=Mutect2,VarDict;CSQ=-|intron_variant|MODIF...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_QR84QJN4,5082,212556


In [15]:
#merge with chr len information
chrom_len=pd.read_csv('/pollard/home/shzhang/akita_variant_scoring/data/chrom_lengths_hg38', sep='\t', header=None)
chrom_len[0]='chr' + chrom_len[0]
simple=simple_top.merge(chrom_len, right_on=0, left_on='CHROM', how='left')
simple = simple.reset_index(drop=True)

simple

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,BiospecimenID,sample_var_index,overall_index,0,1
0,chr10,122357800,.,CCA,C,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intergenic_variant|...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,1452,2425,chr10,133797422
1,chr15,28432799,.,CAT,C,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intergenic_variant|...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,1842,2815,chr15,101991189
2,chr2,32439175,.,TTG,T,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intron_variant|MODI...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A94A7EJ4,1501,4993,chr2,242193529
3,chr3,17630126,.,AAGAGG,A,.,PASS,"CAL=Strelka2,Mutect2,VarDict,Lancet;CSQ=-|intr...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A94A7EJ4,2930,6422,chr3,198295559
4,chr7,158180223,.,C,CAT,.,PASS,"CAL=Strelka2,Mutect2,Lancet;CSQ=AT|intron_vari...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A94A7EJ4,7033,10525,chr7,159345973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,chr9,88136504,.,C,CTT,.,PASS,"CAL=Strelka2,VarDict;CSQ=TT|intron_variant|MOD...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_G6V64VWY,1338,206438,chr9,138394717
138,chr10,46535047,.,G,GCC,.,PASS,"CAL=Strelka2,Mutect2,VarDict;CSQ=CC|intron_var...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_QR84QJN4,3072,210546,chr10,133797422
139,chr15,28646249,.,ACTTTTC,A,.,PASS,"CAL=Mutect2,VarDict;CSQ=-|downstream_gene_vari...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_QR84QJN4,4088,211562,chr15,101991189
140,chrX,85899908,.,TAACAAGTGTTGATGAGGATACAGAAAAAAGCAGACCCTTTACACT...,T,.,PASS,"CAL=Mutect2,VarDict;CSQ=-|intron_variant|MODIF...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_QR84QJN4,5082,212556,chrX,156040895


In [16]:
simple = simple.rename(columns={1: 'chr_len'})


simple['POS_300KB']=np.floor(np.maximum(0, simple['POS']-150e3))
simple['END_300KB']=np.floor(np.minimum(simple['chr_len'], simple['POS']+150e3))
simple['POS_1MB']=np.floor(np.maximum(0, simple['POS']-5e5))
simple['END_1MB']=np.floor(np.minimum(simple['chr_len'], simple['POS']+ 5e5))


simple['postoend_1mb']=simple['END_1MB']-simple['POS_1MB']
simple['postoend_300kb']=simple['END_300KB']-simple['POS_300KB']



In [17]:
simple


,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,BiospecimenID,sample_var_index,overall_index,0,chr_len,POS_300KB,END_300KB,POS_1MB,END_1MB,postoend_1mb,postoend_300kb
0,chr10,122357800,.,CCA,C,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intergenic_variant|...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,1452,2425,chr10,133797422,122207800.0,122507800.0,121857800.0,122857800.0,1000000.0,300000.0
1,chr15,28432799,.,CAT,C,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intergenic_variant|...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_19EJ85F8,1842,2815,chr15,101991189,28282799.0,28582799.0,27932799.0,28932799.0,1000000.0,300000.0
2,chr2,32439175,.,TTG,T,.,PASS,"CAL=Strelka2,VarDict;CSQ=-|intron_variant|MODI...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A94A7EJ4,1501,4993,chr2,242193529,32289175.0,32589175.0,31939175.0,32939175.0,1000000.0,300000.0
3,chr3,17630126,.,AAGAGG,A,.,PASS,"CAL=Strelka2,Mutect2,VarDict,Lancet;CSQ=-|intr...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A94A7EJ4,2930,6422,chr3,198295559,17480126.0,17780126.0,17130126.0,18130126.0,1000000.0,300000.0
4,chr7,158180223,.,C,CAT,.,PASS,"CAL=Strelka2,Mutect2,Lancet;CSQ=AT|intron_vari...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_A94A7EJ4,7033,10525,chr7,159345973,158030223.0,158330223.0,157680223.0,158680223.0,1000000.0,300000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,chr9,88136504,.,C,CTT,.,PASS,"CAL=Strelka2,VarDict;CSQ=TT|intron_variant|MOD...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_G6V64VWY,1338,206438,chr9,138394717,87986504.0,88286504.0,87636504.0,88636504.0,1000000.0,300000.0
138,chr10,46535047,.,G,GCC,.,PASS,"CAL=Strelka2,Mutect2,VarDict;CSQ=CC|intron_var...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_QR84QJN4,3072,210546,chr10,133797422,46385047.0,46685047.0,46035047.0,47035047.0,1000000.0,300000.0
139,chr15,28646249,.,ACTTTTC,A,.,PASS,"CAL=Mutect2,VarDict;CSQ=-|downstream_gene_vari...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_QR84QJN4,4088,211562,chr15,101991189,28496249.0,28796249.0,28146249.0,29146249.0,1000000.0,300000.0
140,chrX,85899908,.,TAACAAGTGTTGATGAGGATACAGAAAAAAGCAGACCCTTTACACT...,T,.,PASS,"CAL=Mutect2,VarDict;CSQ=-|intron_variant|MODIF...",GT:AD:ADC:ADR:AF:AFC:AFR:DP:DPC:DPR:GTC:GT_STATUS,BS_QR84QJN4,5082,212556,chrX,156040895,85749908.0,86049908.0,85399908.0,86399908.0,1000000.0,300000.0


In [22]:
variants_300kb=simple
variants_300kb=variants_300kb[['CHROM', 'POS_300KB', 'END_300KB', 'overall_index']]
variants_300kb['POS_300KB']=variants_300kb['POS_300KB'].astype(int)
variants_300kb['END_300KB']=variants_300kb['END_300KB'].astype(int)
print(len(variants_300kb))
variants_300kb.head()

142


,CHROM,POS_300KB,END_300KB,overall_index
0,chr10,122207800,122507800,2425
1,chr15,28282799,28582799,2815
2,chr2,32289175,32589175,4993
3,chr3,17480126,17780126,6422
4,chr7,158030223,158330223,10525


In [23]:
variants_1mb=simple
variants_1mb=variants_1mb[['CHROM', 'POS_1MB', 'END_1MB', 'overall_index']]
variants_1mb['POS_1MB']=variants_1mb['POS_1MB'].astype(int)
variants_1mb['END_1MB']=variants_1mb['END_1MB'].astype(int)
print(len(variants_1mb))
variants_1mb.head()

142


,CHROM,POS_1MB,END_1MB,overall_index
0,chr10,121857800,122857800,2425
1,chr15,27932799,28932799,2815
2,chr2,31939175,32939175,4993
3,chr3,17130126,18130126,6422
4,chr7,157680223,158680223,10525


In [24]:
variant_300kb_BED = BedTool.from_dataframe(variants_300kb)
variant_1mb_BED = BedTool.from_dataframe(variants_1mb)

In [25]:
synteny_BED = BedTool.from_dataframe(synteny_condensed)


In [26]:
#reports only SVs where there is overlap with  gene
#-F ensures that at least 0.5 of gene has overlap with SV 
#use loj in this case, because many SVs overlap multiple syntenic regions  
variant_300kb_synt=variant_300kb_BED.intersect(synteny_BED, loj=True, wo=True).to_dataframe(header=None)
variant_1mb_synt=variant_1mb_BED.intersect(synteny_BED, loj=True, wo=True).to_dataframe(header=None)


In [27]:
variant_300kb_synt.columns = list(variants_300kb.columns) + list(synteny_condensed.columns)
variant_300kb_synt = variant_300kb_synt[variant_300kb_synt['hg_chr'] != '.']
print(len(variant_300kb_synt))
variant_300kb_synt.head()

16547


,CHROM,POS_300KB,END_300KB,overall_index,hg_chr,hg_start,hg_end,hg_len,hg_seq,mm_chr,mm_start,mm_end,mm_len,mm_seq,total_align_len,hg_dash_n,mm_dash_n,hg_percent,mm_percent
0,chr10,122207800,122507800,2425,chr10,122207147,122208150,1004,aattagccaggcgtggtggcacacacctgtaatcccagctactcgg...,chr7,130725901,130726680,780,AACTAGATGGGCATGGTTGC---------------------CTTG-...,1032,28,252,0.9728682170542635,0.7558139534883721
1,chr10,122207800,122507800,2425,chr10,122209989,122212651,2663,ccctggtggtttattcgctatcttcccctgaagaatgtgacttcca...,chr7,130728155,130730902,2748,CTTTGGTGGC-------------------AATGACAATGGCTGCTA...,2902,239,154,0.9176430048242591,0.9469331495520331
2,chr10,122207800,122507800,2425,chr10,122212878,122214639,1762,AACTTGATTTCAAGCCTGGAATAGGAGAAGTCTGCAATGGCTAGAC...,chr7,130731103,130732753,1651,AACTTTATTTCAAAAGTGGAATAAT---ACTCACAAATGTCTACCC...,1833,71,182,0.96126568466994,0.900709219858156
3,chr10,122207800,122507800,2425,chr10,122214713,122215469,757,CAGCCCCGCTA-TTCATATGGAGG----------AGAAAAAGGGAG...,chr7,130732889,130733584,696,CACCTCCATTACCTTGTATgaaagaacaaaagaaagaaaaaggaag...,797,40,101,0.9498117942283564,0.8732747804265998
4,chr10,122207800,122507800,2425,chr10,122216399,122217427,1029,CATGGTCATTCCCTGAAACAAACCATCATAAAAGGAGATTCATAGA...,chr7,130734765,130735709,945,CATGGTCACTCCCTGAAATCAACCATCAGTAAAGGAGATCCATAGA...,1060,31,115,0.970754716981132,0.8915094339622641


In [28]:
variant_1mb_synt.columns = list(variants_1mb.columns) + list(synteny_condensed.columns)
variant_1mb_synt = variant_1mb_synt[variant_1mb_synt['hg_chr'] != '.']
print(len(variant_1mb_synt))
variant_1mb_synt.head()

54233


,CHROM,POS_1MB,END_1MB,overall_index,hg_chr,hg_start,hg_end,hg_len,hg_seq,mm_chr,mm_start,mm_end,mm_len,mm_seq,total_align_len,hg_dash_n,mm_dash_n,hg_percent,mm_percent
0,chr10,121857800,122857800,2425,chr10,121860593,121861411,819,TGAGAAAGGAAGTACAGTAAATGTTTTTAATAAGT-TGGACAGAAA...,chr7,130472471,130473302,832,TGATAAAGGACTTACCACAAATGGTTTTAATAAATATATACAGAAA...,872,53,40,0.939220,0.954128
1,chr10,121857800,122857800,2425,chr10,121861835,121862235,401,CCCTCATGACTAGGGTTAACAACAACAATGTACTCCTATTTTCTGC...,chr7,130473648,130474021,374,TCCTTACATATACAGCTACAAGAAACAAAACCCTGCTCCTCTCTGG...,401,0,27,1.000000,0.932668
2,chr10,121857800,122857800,2425,chr10,121862273,121862391,119,ACAGCCATTCCTTCTGAATTCTAGCCATTGGAGAGACCTTGATATC...,chr7,130474043,130474146,104,ACCACTTCTCTTTCCAAATTCTGGCCGTTAGCAATACCTTACCACA...,119,0,15,1.000000,0.873950
3,chr10,121857800,122857800,2425,chr10,121864539,121866810,2272,CAGAAACAGAAATTACCATGAAGCAGAGTTTTCACttagaccattt...,chr7,130475811,130477803,1993,CAAAAATAAAATTTCCCATGCAGCCAAGTTTG-ATTTAGGCT-TTT...,2382,110,389,0.953820,0.836692
4,chr10,121857800,122857800,2425,chr10,121866931,121867299,369,AACTGAGAGCATCCGAAGATTACAAAGGACAAAGCTCTCAATCTTA...,chr7,130477804,130478171,368,AACTAATAACatccaaaggt--aaaaagacagagcttctggtctca...,403,34,35,0.915633,0.913151


In [31]:
def find_index(hg_seq, idx):
    count_letters = 0  # To count the letters encountered in the combined string
    idx_new = 0  # Corresponding index in combined string
    
    for char in hg_seq:
        if count_letters == (idx+1):
            break
        if char.isalpha():
            count_letters += 1
        idx_new += 1
    
    return idx_new

#given two aligned strings with start and end idx
#count up amount of same characters within the alignment subset 
def count_str(hg_seq, mm_seq, start_idx, end_idx):
    #print('start_idx', start_idx)
    #print('end_idx', end_idx)
    hg_subset = hg_seq[start_idx:end_idx]
    mm_subset = mm_seq[start_idx:end_idx]

    count_same = sum(1 for a, b in zip(hg_subset, mm_subset) if a == b)
    
#     if end_idx>start_idx:
#         prop=count_same/(end_idx-start_idx)
#     else:
#         prop=0
    prop=count_same/(end_idx-start_idx)

    return(prop)

In [33]:
start=time.time()
for i in simple.index:
   
    if i % 10 == 0:
        print('index ', i)
    
    
    #THIS ONE FOR 1MB SCORE 
    try:
        #print(i)
        variant=simple.loc[i]
        idx=variant.overall_index
        int_df=variant_1mb_synt[variant_1mb_synt['overall_index'] == idx]

        #print(len(int_df))
    
    
        if len(int_df)>0:


            int_df['start_idx']=(np.maximum(int_df['POS_1MB'], int_df['hg_start'])) - int_df['hg_start'] 
            int_df['end_idx']=(np.minimum(int_df['END_1MB'], int_df['hg_end'])) - int_df['hg_start'] 
            int_df['n_overlap']=int_df['end_idx']-int_df['start_idx']

            
            #get alignment indices
            int_df['start_align_idx'] = int_df.apply(lambda row: find_index(row['hg_seq'], row['start_idx']), axis=1)
            int_df['end_align_idx'] = int_df.apply(lambda row: find_index(row['hg_seq'], row['end_idx']), axis=1)
            int_df['n_align_overlap']=int_df['end_align_idx']-int_df['start_align_idx']
        
        
            int_df['count_prop'] = int_df.apply(lambda row: count_str(row['hg_seq'], row['mm_seq'], row['start_align_idx'], row['end_align_idx']), axis=1)


            #proportion of SV that covers a syntenic region  
            simple.loc[i, 'synt_prop_overlap_1MB'] = int(np.sum(int_df['n_overlap'])) /abs(simple.loc[i, 'postoend_1mb'])
            #weighted proportion of SV in syntenic region, and how similar those regions are 
            simple.loc[i, 'align_synt_prop_1MB'] =  (np.sum(int_df['count_prop'] *int_df['n_overlap'])) /abs(simple.loc[i, 'postoend_1mb'])



        else:
            simple.loc[i, 'synt_prop_overlap_1MB'] = 0
            simple.loc[i, 'align_synt_prop_1MB'] = 0
            
    except:
        print('error in index', i)
        
        
    #THIS ONE FOR NORMAL SV LENGTH 
    try:
        #print(i)
        variant=simple.loc[i]
        idx=variant.overall_index
        int_df=variant_300kb_synt[variant_300kb_synt['overall_index'] == idx]
        #print(len(int_df))
    
    
        if len(int_df)>0:


            int_df['start_idx']=(np.maximum(int_df['POS_300KB'], int_df['hg_start'])) - int_df['hg_start'] 
            int_df['end_idx']=(np.minimum(int_df['END_300KB'], int_df['hg_end'])) - int_df['hg_start'] 
            int_df['n_overlap']=int_df['end_idx']-int_df['start_idx']

            
            #get alignment indices
            int_df['start_align_idx'] = int_df.apply(lambda row: find_index(row['hg_seq'], row['start_idx']), axis=1)
            int_df['end_align_idx'] = int_df.apply(lambda row: find_index(row['hg_seq'], row['end_idx']), axis=1)
            int_df['n_align_overlap']=int_df['end_align_idx']-int_df['start_align_idx']
        
        
            int_df['count_prop'] = int_df.apply(lambda row: count_str(row['hg_seq'], row['mm_seq'], row['start_align_idx'], row['end_align_idx']), axis=1)


            #proportion of SV that covers a syntenic region  
            simple.loc[i, 'synt_prop_overlap_300KB'] = int(np.sum(int_df['n_overlap'])) /abs(simple.loc[i, 'postoend_300kb'])
            #weighted proportion of SV in syntenic region, and how similar those regions are 
            simple.loc[i, 'align_synt_prop_300KB'] =  (np.sum(int_df['count_prop'] *int_df['n_overlap'])) /abs(simple.loc[i, 'postoend_300kb'])



        else:
            simple.loc[i, 'synt_prop_overlap_300KB'] = 0
            simple.loc[i, 'align_synt_prop_300KB'] = 0
            
    except:
        print('error 1mb in index', i)
        
        
end=time.time()
print(f"Time taken: {end-start} seconds")
    

index  0
index  10
index  20
index  30
index  40
index  50
index  60
index  70
index  80
index  90
index  100
index  110
index  120
index  130
index  140
Time taken: 24.575964212417603 seconds


In [34]:
simple.to_csv('/pollard/home/shzhang/20231024_simple_topvariants_synteny_combined.txt', sep='\t', index=False)  

In [ ]:
simple